In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [8]:
def quaternion(q):
    q = np.array([q[0],q[1],q[2],q[3]])
    return q

def q_conj(q):
    q = np.array([q[0],-1.*q[1],-1.*q[2],-1.*q[3]])
    return q

def q_mult(a,b):
    mult_val = [a[0]*b[0] - a[1]*b[1] - a[2]*b[2] - a[3]*b[3],
                a[0]*b[1] + a[1]*b[0] + a[2]*b[3] - a[3]*b[2],
                a[0]*b[2] - a[1]*b[3] + a[2]*b[0] + a[3]*b[1],
                a[0]*b[3] + a[1]*b[2] - a[2]*b[1] + a[3]*b[0]]
    return np.transpose(np.array(mult_val))

def normalize(q):
    q_norm = np.sum([x**2 for x in q])
    return q/q_norm

def IMU_update(q,w,a,beta=1.,freq=1/256):
    F = np.array([
                  2.*(q[1]*q[3] - q[0]*q[2]) - a[0],
                  2.*(q[0]*q[1] + q[2]*q[3]) - a[1],
                  2.*(0.5 - q[1]**2 - q[2]**2) - a[2]
                 ])
    
    J = np.array([
         [-2.*q[2],  2.*q[3], -2.*q[0], 2.*q[1]],
         [ 2.*q[1],  2.*q[0],  2.*q[3], 2.*q[2]],
         [      0., -4.*q[1], -4.*q[2], 0.     ]
        ])
    
    step = np.matmul(np.transpose(J),F)
    step = normalize(step)
    
    q_w = np.array([0,w[0],w[1],w[2]])
    q_dot = 0.5*(q_mult(q,q_w)) - beta*np.transpose(step)
    q = q + freq*q_dot
    q = normalize(q)
    return q